<a href="https://colab.research.google.com/github/ebatty/IntroCompNeuro/blob/main/lectures/04_NeuralEncodingI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 - Neural Encoding I

Learning objectives of lecture/notes: After lecture, students should be able to:

- Describe structure of linear-nonlinear-Poisson model

- Explain how to determine whether a neuron is Poisson-like


In [ ]:
# @markdown Imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 20})

# Section 1: Encoding models





<img src="https://raw.githubusercontent.com/ebatty/IntroCompNeuro/main/images/NeuralCoding.jpg">

In order to build a neural encoding model, we want to be able to predict the spikes of a neuron given the stimulus or behavior. So far, with tuning curves and spike triggered averages, we have explored how to understand more about the relationship between stimulus and neural response, but we have not yet built a full encoding model that allows us to predict spikes.

The idea behind an encoding model is that you determine the general structure, basically a set of equations of how the stimulus is converted in to the spikes of the neuron. These equations involve learnable variables, or parameters. You learn these parameters from neural data to fit your encoding model. We'll touch much more on model fitting in future days. For now, we will focus on the structure of the most common type of encoding model: the linear-nonlinear-Poisson model.

# Section 2: Linear-nonlinear-Poisson models

Linear-nonlinear-Poisson (LNP) models consist of a linear filtering stage of the stimulus, then a point-wise nonlinearity, then draws of spikes from the predicted firing rate, assuming a Poisson distribution. The schematic below outlines these steps. The LNP model can also be described with the following equations:

$$\lambda_t = f(x_t \cdot k) \\
y_t \sim
 Poiss(\lambda_t)$$

where $x_t$ is the stimulus we want to use to predict the neural reponse at time bin $t$, $k$ is the linear filter, $\lambda_t$ is the firing rate of the neuron in spikes per second, $y_t$ is the spike count at time bin $t$, and $f$ is some nonlinearity.

We will dive into each of these stages in the following sections.

<img src="https://raw.githubusercontent.com/ebatty/IntroCompNeuro/main/images/LNP.png" alt="STA from movie">

## Section 2.1: Linear filtering

Let's return to our spike-triggered average from the last lecture. Once you compute the spike-triggered average, can you use it to start to predict whether the neuron will fire or not? Remember that the spike-triggered average is the average stimulus that triggers a spike. Intuitively, if a chunk of stimulus is more like the spike-triggered average, the neuron is more likely to fire. If it very dissimilar to the spike-triggered average, the neuron will probably not fire.

So we need a way to compute how similar a chunk of stimulus is to the spike-triggered average. We will do this using the **dot product** between that chunk of stimulus and the spike-triggered average. We will compute $x_t \cdot STA$ where $x_t$ is whatever you are using to predict the neural response at time bin $t$, in this case the the stimulus from let's say 10 bins before $t$ to time bin $t$.

We have motivated this using the spike-triggered average, but for now let's return to referring to our general linear filter $k$. This linear filter $k$ is what we will learn from data - it's the estimate of the neuron's linear receptive field within the context of this encoding model. You can think of this as similar to the STA - in fact, for some stimuli, the STA is a pretty good estimate for this linear filter! However, it isn't always so they are distinct concepts. So once we learn our linear filter from data(which we will cover next class), the first stage to predicting our neural spikes is to take the dot product of this linear filter with the relevant stimulus features: $x_t \cdot k$.


In Sections 2.1.1 and 2.1.2 below, we dive into why we use the dot product for this. In Section 2.1.3, we dive further into what $x_t$ could be (stimulus over time and/or different stimulus features).


### Section 2.1.1: Dot product as neural computation

Why do we use the dot product for this linear filtering stage?  One simple reason is that neurons are essentially computing dot products of inputs - a model of how neural processing works is probably better if it takes into account how that processing can occur! 

Let's look at the example of a retinal ganglion cell - a neuron in the retina (the back of the eye). Note that the following description is quite a bit of an oversimplification for illustrative purposes. Retinal ganglion cells receive inputs from photoreceptors. Each photoreceptor responds more when there is light in a particular part of the visual field - the receptive field of that photoreceptor. The photoreceptor response is thus kind of like the value of a pixel in an grayscale image: higher values indicate white, lower values indicate black. Axons from multiple photoreceptors synapse onto a single retinal ganglion cell. The response of that retinal ganglion cell is basically a weighted sum of the photoreceptor responses, where the weights are the strengths of the synapses from each photoreceptor. In other words, the retinal ganglion cell is computing the dot product between a vector of its synapse weights and a vector of the photoreceptor responses for a certain image. Similarly, in our linear filtering stage, we could have $x_t$ include the intensities of various pixels in an image, and the linear filter $k$ is essentially the weight from each pixel, a very similar computation.

### Section 2.1.2: Dot product as measure of similarity

Dot products give a rough sense of similarity between the linear filter and the stimulus.  To see why, we'll pretend we're predicting the neural response at a time bin $t$ based on the 1d stimulus at the preceding time step $s_{t-1}$ and at the current time step $s_{t}$. This means our linear filter $k$ has two entries. We already know our linear filter: $k = [1, 1]$. We want to predict whether a certain two time steps of the stimulus are likely to result in the neuron spiking. Remember, we want to look at whether that chunk of stimulus is similar to the linear filter.

Below, I plot our stimulus data. Each dot corresponds to a different two time step chunk of the stimulus: the prior time step is the x axis and the current time step is the y axis. We represent the linear filter $k$ on this plot as well, in red. 





In [ ]:
# @markdown Execute to visualize data
np.random.seed(123)

data = np.random.randn(20, 2)

fig, ax = plt.subplots(1, 1, figsize = (10, 10))


ax.plot(data[:,0], data[:, 1], 'ob', label = 'Data (x)')
ax.plot(1, 1, 'ro', label = 'Linear filter (k)')
ax.plot([-3, 3], [0, 0], 'k-')
ax.plot([0, 0], [-3, 3], 'k-')
ax.set_frame_on(False)
ax.set(yticks = [], yticklabels = [],
       xticks = [], xticklabels = []);
ax.legend(frameon=False)
ax.annotate(r"$s_t$", (0.1, 3))
ax.annotate(r"$s_{t-1}$", (3, .1));



If you recall, another way of defining the dot product of two vectors, in this case $x_t$ and $k$, is as the length of the first multiplied with the length of the second, multiplied with the angle between them:

$$x_t \cdot k = ||x_t|| ||k|| cos(\theta) $$

The length of $k$ would be constant for every time bin/new stimulus we look at, so we can basically ignore it. The dot product basically then depends on the length of the stimulus vector and the cosine of the angle between them. 

The cosine of the angle between two vectors is a measure of their similarity. If the angle is 0, the two vectors lie in the same direction so are very similar and the cosine of that angle is 1. If the angle is 180, the two vectors point in opposite directions and are very dissimilar: in this case the cosine of this angle is -1. In between these two extremes, the cosine of the angle moves between 1 and -1 as the angle between vectors grows larger. 

So, the dot product is basically a measure of similarity (the cosine of the angle between vectors) times the length of the vector $x_t$. So it is not exactly a measure of similarity, because of this last term, but is close to one.

In [ ]:
# @markdown Execute to visualize vectors
fig, ax = plt.subplots(1, 1, figsize = (10, 10))

ax.plot([0, data[2,0]], [0, data[2, 1]], '-ob', label = 'Data (x)')
ax.plot([0, 1], [0, 1], '-ro', label = 'Linear filter (k)')
ax.plot([-3, 3], [0, 0], 'k-')
ax.plot([0, 0], [-3, 3], 'k-')
ax.set_frame_on(False)
ax.set(yticks = [], yticklabels = [],
       xticks = [], xticklabels = []);
ax.legend(frameon=False)
ax.annotate(r"$s_t$", (0.1, 3))
ax.annotate(r"$s_{t-1}$", (3, .1));
ax.annotate(r"$\theta$", (.1, .5));

Another way to think of this computation is as the scalar projection of the data points onto the linear filter vector. Let's define the data point $x_t$ as the addition of two vectors: one along the linear filter direction (green vector below) and the other perpendicular to that (purple vector below). The scalar projection is then the length of the first vector (green vector) along the linear filter direction. The diagram below will probably make this clearer.

The scalar projection of $x_t$ onto $k$ is equal to the length of $x_t$ times the cosine of the angle $\theta$. In other words, the dot product of $x_t$ with $k$, divided by the length of $k$. Since we are ignoring the length of $k$ as that will remain constant, the dot product is basically computing the scalar projection!


In [ ]:
# @markdown Execute to visualize vectors
fig, ax = plt.subplots(1, 1, figsize = (10, 10))

ax.plot([0, data[2,0]], [0, data[2, 1]], '-ob', label = 'Data (x)')
ax.plot([0, 1], [0, 1], '-ro', label = 'Linear filter (k)')
k = np.array([1, 1])
scalar_proj = np.dot(data[2], k)/np.linalg.norm(k)
vec_proj = scalar_proj * k/np.linalg.norm(k)
ax.plot([0, vec_proj[0]], [0, vec_proj[1]], '-go')
ax.plot([data[2,0], vec_proj[0]], [data[2,1], vec_proj[1]], '-o', color = 'purple')

ax.plot([-3, 3], [0, 0], 'k-')
ax.plot([0, 0], [-3, 3], 'k-')
ax.set_frame_on(False)
ax.set(yticks = [], yticklabels = [],
       xticks = [], xticklabels = []);
ax.legend(frameon=False)
ax.annotate(r"$s_t$", (0.1, 3))
ax.annotate(r"$s_{t-1}$", (3, .1));
ax.annotate(r"$\theta$", (.1, .5));

### Section 2.1.2: Filtering over time or stimulus/behavior features

So what is $x_t$? 

Bit of a trick question because $x_t$ is whatever you choose it to be. Essentially, as the researcher, you choose what parts of the stimulus or behavior you want to use to predict the neural response at time bin $t$. Let's say you have a 1d stimulus over time and want to use the previous 10 time bins of the stimulus for prediction. In that case, $x_t$ would the stimulus for the 10 time bins prior, a vector with 10 elements. The linear filter would also be a vector with 10 elements. We would call this filter a temporal filter (it's acting over time)

Let's say you wanted to predict the neural response based just on the frame of a movie at the current time bin. In that case $x_t$ would be all the pixel values in the image. Note that an image is 2-dimensional so we want to flatten this into a vector. In this case the linear filter would be a vector with length equal to the total number of pixels. We would call this filter a spatial filter (it's acting over space)





**Stop and think!** You could go even more complicated and use the previous 10 frames of a movie. If each frame was 5 pixels by 5 pixels, what length would your linear filter be?

We can put every feature that we want to use to predict as an entry in our vector $x_t$. In this case that would be 5 x 5 x 10 = 250 features. The linear filter would be the same length so would have 250 entries. 

We would call the filter in the example above a spatiotemporal filter (it's acting over both space and time).

## Section 2.2 Nonlinearity

The output of the linear filtering stage would not be a good prediction for the firing rate (in spikes/bin) of our neuron. It is the output of a dot product so could be below 0 or a huge number. We know that the firing rate of a neuron can't go below 0 (there can't be a negative number of spikes per second) and has some max value (due to the refactory period of a neuron, the neuron can't fire an unlimited amount in a second). Additionally, neurons do tend to do some nonlinear processing of their inputs - they are not just firing in proportion to the summation of their inputs. 

Due to these reasons, we put the output of the linear filtering stage, a single number for each time bin, through a nonlinearity. 

$$\lambda_t = f(x_t \cdot k)$$

where $x_t$ is the features we use for prediction, $k$ is the linear filter, $\lambda_t$ is the firing rate in spikes/bin, and $f$ is our nonlinearity.


We can choose what nonlinearity to use, or we could fit our nonlinearity to data. Researchers often choose an exponential nonlinearity for reasons that will be explained in the next lecture. Note that while this prevents negative firing rates, it doesn't actually bound them under a maximum! Despite this, it has worked fairly well in practice. 

Let's go through a numerical example of the model so far. Our stimulus is:

s = [5, -1, 4, 0, 2]

Each entry is the value of the 1d stimulus at a time bin.

Our linear filter for the 3 time bins up to and including the time bin of a spike is:

k = [.3, 1, .4]

We are using an exponential nonlinearity.



 


**Stop and think!** What is the predicted firing rate, $\lambda$ for time bins 3 and after based on the above numbers? 

\begin{align}
\lambda_3 &= e^{x_3 \cdot k}\\
&= e^{[5, -1, 4] \cdot [.3, 1, .4]}\\
&= e^{2.1} \\
&= 8.16
\end{align}

\begin{align}
\lambda_4 &= e^{x_4 \cdot k}\\
&= e^{[-1, 4, 0] \cdot [.3, 1, .4]}\\
&= e^{3.7} \\
&= 40.45
\end{align}

\begin{align}
\lambda_5 &= e^{x_4 \cdot k}\\
&= e^{[4, 0, 2] \cdot [.3, 1, .4]}\\
&= e^{2} \\
&= 7.38
\end{align}

So, we're predicting a firing rate in terms of spikes per bins of 

[0, 0, 8.16, 40.45, 7.38]. 

Note we just predicted zero for the first two time bins because we don't have enough preceding stimulus to use for $x_t$. There are better methods for this edge cases but unfortunately, that's outside of the scope of this lecture.


## Section 2.3: Poisson stage

We can now predict firing rate using a linear filtering stage followed by a nonlinearity. However, we want to go all the way to the actual data we're collecting - spikes. We will make a very specific assumption about spikes in this model: we will assume that the spike count in a time bin (the number of spikes in that time bin) is drawn from a probability distribution that depends on the firing rate. This means that the same firing rate can result in different predictions of spike counts, as we have introduced randomness.

Remember our raster plots (one shown below)? These are showing the spikes in each trial. Each trial correponds to one presentation of the same stimulus. If neural responses depended entirely on the stimulus and there was no randomness in neural spiking, each trial would result in the exact same number of spikes at the exact same times. This is clearly not the case. This linear-nonlinear-Poisson model assumes that there is the same underlying firing rate on each trial but that the exact spikes seen are slightly different because they're being drawn from a probability distribution that is based on the firing rate.

<img src="https://praneethnamburi.files.wordpress.com/2015/02/02_raster_baselineandstim.png" alt="rasterplotexample">

In particular, we use the Poisson distribution to account for neural spiking. The Poisson distribution is defined as 

$$p(y_t | \lambda_t) = \frac{\lambda_t^{y_t} e^{-\lambda_t}}{y_t!} $$

where 

$y_t$: spike count for bin $t$

$\lambda_t$: firing rate in terms of spikes per bin

$p(y_t | \lambda_t)$: the probability of the spike count being equal to some number given the firing rate

Let's say we have $\lambda_t$ as .4. 

The probability of seeing 0 spikes is 
$p(y_t = 0 | \lambda_t = 0.4)$ = 0.67, as computed in the code below. We can do the same computation to find that the probability of seeing 1 spike is .27, the probability of seeing 2 spikes is 0.05, and the probability of seeing 3 spikes is 0.007. If we drew a spike count from a Poisson distribution with $\lambda_t = 0.4$, we would get zero spikes 67% of the time, 1 spike 27% of the time, and so on.

Different $\lambda_t$'s would affect these probabilities: higher $\lambda_t$ would result in higher spike counts, which makes sense as $\lambda_t$ is our firing rate.


In [ ]:
(.4**0 * np.exp(-.4))/np.math.factorial(0)

Why do we use the Poisson distribution for neural spiking? It works nicely because the Poisson distribution is only defined for non-negative integers. Spike counts are non-negative integers, so this is a natural fit. However, we do have to look at if neurons fire spikes in a Poisson-like way. This isn't guaranteed! It turns out that the Poisson distribution seems to be a fairly good way to capture spiking responses, although it doesn't work well for all neurons. Dayan & Abbott 2001 covers this comparison to data well: read the section on Comparison with Data from page 31 to 24 for more information: http://www.gatsby.ucl.ac.uk/~lmate/biblio/dayanabbott.pdf

Coding note: we can draw random spikes using a Poisson distribution as shown in the code below. Remember our firing rates from a previous problem of [0, 0, 8.16, 40.45, 7.38].

In [ ]:
# Create firing rates
firing_rates = np.array([0, 0, 8.16, 40.45, 7.38])

# Draw spike counts based on firing rates
spikes = np.random.poisson(firing_rates)

print(spikes)

# Summary

We now have a model that takes in the stimulus and predicts spikes. In the next lecture, we will cover why this model is popular, how to fit it to data (learn what the linear filter is), and how to use it to learn about neural processing.

# Optional Reading

Notes on Generalized Linear Models (LNPs): https://arxiv.org/pdf/1404.1999.pdf

Video diving further into these types of models:  https://youtube.com/watch?v=NXVG9ORBYXQ 



